In [1]:
import tensorflow as tf
from keras import layers, models, optimizers, losses
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd, numpy as np

2025-11-09 23:10:59.549123: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
X_train = tf.convert_to_tensor(
    pd.read_csv("./data/x_train.csv").to_numpy(), dtype=float
)
y_train = tf.convert_to_tensor(
    pd.read_csv("./data/y_train.csv").to_numpy(), dtype=float
)

X_test = tf.convert_to_tensor(pd.read_csv("./data/x_test.csv").to_numpy(), dtype=float)
y_test = tf.convert_to_tensor(pd.read_csv("./data/y_test.csv").to_numpy(), dtype=float)

2025-11-09 23:11:03.552639: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [3]:
# ------------------------------------------
# DEFINE THE MODEL
# ------------------------------------------

# Using Sequential model — a simple stack of layers
model = models.Sequential(
    [
        layers.Input(
            shape=(X_train.shape[1],)
        ),  # Input layer: expects 7 input features
        layers.Dense(1),  # Output layer: 1 neuron (for single output)
    ]
)

# (So the model learns a simple linear relationship: y = W*x + b)

In [4]:
# ------------------------------------------
# COMPILE THE MODEL
# ------------------------------------------

# Specify how the model should learn:
# - optimizer: how to update weights (SGD = Stochastic Gradient Descent)
# - loss: how to measure errors (Mean Squared Error)
model.compile(
    optimizer=optimizers.SGD(learning_rate=0.01), loss=losses.MeanSquaredError()
)

In [5]:
# ------------------------------------------
# TRAIN THE MODEL
# ------------------------------------------

# Train for 500 epochs (passes over the entire dataset)
# batch_size=32 means the model updates weights after every 32 samples
# verbose=0 means “don’t print progress output”
model.fit(X_train, y_train, epochs=500, batch_size=32, verbose=1)

Epoch 1/500
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1640.6732  
Epoch 2/500
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 689us/step - loss: 564.3107
Epoch 3/500
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 746us/step - loss: 494.9800
Epoch 4/500
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 642us/step - loss: 461.0805
Epoch 5/500
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 656us/step - loss: 443.7813
Epoch 6/500
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 654us/step - loss: 434.3430
Epoch 7/500
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 677us/step - loss: 429.7060
Epoch 8/500
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 629us/step - loss: 426.9846
Epoch 9/500
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 675us/step - loss: 425.0068
Epoch 10/500
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 648us/step - loss: 423.9394
Epoch 11/500
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 640us/step - loss: 423.2138
Epoch 12/500
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 630us/step - loss: 422.5372
Epoch 13/500
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 635us/step - loss: 422.1017
Epoch 14/500
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 615us/step - l

In [6]:
# ------------------------------------------
# EVALUATE MODEL PERFORMANCE
# ------------------------------------------

# Evaluate the final loss (error) on the same training data
loss = model.evaluate(X_train, y_train, verbose=0)

# Print the loss value
print(f"Final loss: {loss:.4f}")

# Print the learned weights and bias from the Dense layer
# model.layers[0].get_weights() returns [weights_matrix, bias_vector]
print("Weights:", model.layers[0].get_weights())

Final loss: 419.9782
Weights: [array([[ 2.170903  ],
       [12.129986  ],
       [-0.59762144],
       [ 1.2058551 ],
       [ 1.3573456 ],
       [-0.7668007 ],
       [ 1.7750947 ]], dtype=float32), array([86.71319], dtype=float32)]


In [7]:
model.save("./model/model.keras")

In [8]:
model = models.load_model("./model/model.keras")

In [9]:
y_pred = model.predict(X_test, verbose=0)

y_test_flat, y_pred_flat = y_test.numpy().flatten(), y_pred

mse = mean_squared_error(y_test_flat, y_pred_flat)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_flat, y_pred_flat)
r2 = r2_score(y_test_flat, y_pred_flat)

print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"R2 Score: {r2:.4f}")

Mean Squared Error (MSE): 410.7801
Root Mean Squared Error (RMSE): 20.2677
Mean Absolute Error (MAE): 16.3657
R2 Score: 0.3127
